In [1]:
import pandas as pd
import numpy as np
from pandas.core.interchange.from_dataframe import categorical_column_to_series

In [6]:
chunk_size = 100000

In [5]:
# Initialize variables for analysis
total_rows = 0
column_sums = None
column_squared_sums = None

In [6]:
# Step 3: Read the data in chunks
def read_csv_in_chunks(file_path, chunk_size):
    for chunk in pd.read_csv(file_path, chunksize=chunk_size):
        yield chunk

In [8]:
# Step 4: Sample 20% of observation
labels = pd.read_csv('data/train_labels.csv')
sample_labels = labels.sample(frac=0.2, random_state=42)

In [10]:
# Read data
sampled_data = []
for chunk in read_csv_in_chunks('data/train_data.csv', chunk_size):
    merged_chunk = pd.merge(chunk, sample_labels, on='customer_ID', how='inner')
    sampled_data.append(merged_chunk)
    
# Combine all chunks into a single dataframe
development_sample = pd.concat(sampled_data, ignore_index=True)

# Save the development sample
development_sample.to_csv('data/development_sample.csv', index=False)

In [8]:
# load development_sample
data = []
for chunk in read_csv_in_chunks('data/development_sample.csv', chunk_size):
    data.append(chunk)

    

In [13]:
df = pd.concat(data, ignore_index=True)
df.head()

,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,...,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145,target
0,000098081fde4fd64bc4d503a5d6f86a0aedc425c96f52...,2017-03-11,0.374606,0.033519,0.044293,1.008622,0.001470,0.459235,0.002339,0.006168,...,NaN,NaN,0.008263,0.006609,0.007370,NaN,0.007171,0.005120,0.007513,0
1,000098081fde4fd64bc4d503a5d6f86a0aedc425c96f52...,2017-04-22,0.414269,0.002516,0.059667,0.123964,0.004374,0.434148,0.001405,0.052130,...,NaN,NaN,0.001986,0.004050,0.000796,NaN,0.001802,0.002364,0.003987,0
2,000098081fde4fd64bc4d503a5d6f86a0aedc425c96f52...,2017-05-12,0.413310,0.003285,0.053418,0.304955,0.002316,0.415906,0.009388,0.048780,...,NaN,NaN,0.009515,0.008757,0.009219,NaN,0.003134,0.001686,0.001265,0
3,000098081fde4fd64bc4d503a5d6f86a0aedc425c96f52...,2017-06-10,0.328983,0.038574,0.049463,0.115654,0.004654,0.416112,0.003223,0.081001,...,NaN,NaN,0.002524,0.007841,0.007421,NaN,0.000728,0.003591,0.007998,0
4,000098081fde4fd64bc4d503a5d6f86a0aedc425c96f52...,2017-07-19,0.496989,0.005552,0.041452,0.133631,0.007363,0.419864,0.003393,0.098308,...,NaN,NaN,0.003823,0.009599,0.006957,NaN,0.008746,0.007101,0.006658,0


In [26]:
# Step 5: One-hot encoding for categorical variables
# List of known categorical columns
# List of categorical columns
categorical_columns = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']

# Function to read and process data in chunks
def process_chunks(file_path, chunk_size=100000):
    encoded_chunks = []
    
    for chunk in pd.read_csv(file_path, chunksize=chunk_size):
        # Convert all specified columns to strings
        for col in categorical_columns:
            chunk[col] = chunk[col].astype(str)
        
        # Perform one-hot encoding
        encoded_chunk = pd.get_dummies(chunk, columns=categorical_columns)
        
        # Ensure the encoded columns are of type int (0 or 1)
        for col in encoded_chunk.columns:
            if col.startswith(tuple(categorical_columns)):
                encoded_chunk[col] = encoded_chunk[col].astype(int)
        
        encoded_chunks.append(encoded_chunk)
        
        print(f"Processed chunk of size {len(chunk)}")
    
    return pd.concat(encoded_chunks, ignore_index=True)

# Process the file
df_encoded = process_chunks('data/development_sample.csv')

print("One-hot encoding completed.")
print(f"Shape of encoded DataFrame: {df_encoded.shape}")

# Display the first few rows of the encoded DataFrame
print(df_encoded.head())

# Check the data types of the encoded columns
print(df_encoded.dtypes)

Processed chunk of size 100000
Processed chunk of size 100000
Processed chunk of size 100000
Processed chunk of size 100000
Processed chunk of size 100000
Processed chunk of size 100000
Processed chunk of size 100000
Processed chunk of size 100000
Processed chunk of size 100000
Processed chunk of size 100000
Processed chunk of size 100000
Processed chunk of size 7082
One-hot encoding completed.
Shape of encoded DataFrame: (1107082, 235)
                                         customer_ID         S_2       P_2  \
0  000098081fde4fd64bc4d503a5d6f86a0aedc425c96f52...  2017-03-11  0.374606   
1  000098081fde4fd64bc4d503a5d6f86a0aedc425c96f52...  2017-04-22  0.414269   
2  000098081fde4fd64bc4d503a5d6f86a0aedc425c96f52...  2017-05-12  0.413310   
3  000098081fde4fd64bc4d503a5d6f86a0aedc425c96f52...  2017-06-10  0.328983   
4  000098081fde4fd64bc4d503a5d6f86a0aedc425c96f52...  2017-07-19  0.496989   

       D_39       B_1       B_2       R_1       S_3      D_41       B_3  ...  \
0  0.03351

In [27]:
df_encoded.to_csv('data/train_encoded_data.csv', index=False)

***USE THE BELOW CODE FOR QUESTION 6~***

In [2]:
# Step 6: EDA
df = pd.read_csv('data/train_encoded_data.csv')


In [3]:
df['S_2'] = pd.to_datetime(df['S_2'])

In [36]:
df.head()

,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,...,D_66_1.0,D_66_nan,D_68_0.0,D_68_1.0,D_68_2.0,D_68_3.0,D_68_4.0,D_68_5.0,D_68_6.0,D_68_nan
0,000098081fde4fd64bc4d503a5d6f86a0aedc425c96f52...,2017-03-11,0.374606,0.033519,0.044293,1.008622,0.001470,0.459235,0.002339,0.006168,...,0,1,0,0,0,0,0,0,0,1
1,000098081fde4fd64bc4d503a5d6f86a0aedc425c96f52...,2017-04-22,0.414269,0.002516,0.059667,0.123964,0.004374,0.434148,0.001405,0.052130,...,0,1,0,0,0,0,0,0,0,1
2,000098081fde4fd64bc4d503a5d6f86a0aedc425c96f52...,2017-05-12,0.413310,0.003285,0.053418,0.304955,0.002316,0.415906,0.009388,0.048780,...,0,1,0,0,1,0,0,0,0,0
3,000098081fde4fd64bc4d503a5d6f86a0aedc425c96f52...,2017-06-10,0.328983,0.038574,0.049463,0.115654,0.004654,0.416112,0.003223,0.081001,...,0,1,0,0,1,0,0,0,0,0
4,000098081fde4fd64bc4d503a5d6f86a0aedc425c96f52...,2017-07-19,0.496989,0.005552,0.041452,0.133631,0.007363,0.419864,0.003393,0.098308,...,0,1,0,0,1,0,0,0,0,0


In [9]:
# calculate last 6 months average value for numerical columns
def calculate_average_last_6_months(df):
    end_date = df['S_2'].max()
    start_date = end_date - pd.DateOffset(months=6)
    
    # Filter data for the last 6 months
    last_6_months_data = df[(df['S_2'] >= start_date) & (df['S_2'] <= end_date)]
    
    # Calculate the average for each numerical column
    averages = last_6_months_data.mean(numeric_only=True)
    return averages

# calculate last 12 months average value for numerical columns
def calculate_average_last_12_months(df):
    end_date = df['S_2'].max()
    start_date = end_date - pd.DateOffset(months=12)
    last_12_months_data = df[df['S_2'] >= start_date]
    averages = last_12_months_data.mean(numeric_only=True)
    return averages

# calculate last 6 months minimum value for numerical columns
def calculate_min_last_6_months(df):
    end_date = df['S_2'].max()
    start_date = end_date - pd.DateOffset(months=6)
    last_6_months_data = df[(df['S_2'] >= start_date) & (df['S_2'] <= end_date)]
    minimums = last_6_months_data.min(numeric_only=True)
    return minimums

# calculate last 9 months maximum value for numerical columns
def calculate_max_last_9_months(df):
    end_date = df['S_2'].max()
    start_date = end_date - pd.DateOffset(months=9)
    last_9_months_data = df[(df['S_2'] >= start_date) & (df['S_2'] <= end_date)]
    maximums = last_9_months_data.max(numeric_only=True)
    return maximums

# calculate last 3 months total value for numerical columns
def calculate_sum_of_3_months(df):
    end_date = df['S_2'].max()
    start_date = end_date - pd.DateOffset(months=3)
    last_3_months_data = df[(df['S_2'] >= start_date) & (df['S_2'] <= end_date)]
    sums = last_3_months_data.sum(numeric_only=True)
    return sums

In [5]:
eda_df = df.copy()

In [6]:
# Remove one-hot encoding variables
drop_prefixes = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']

regex_pattern = '^(' + '|'.join(drop_prefixes) + ')'

eda_df = eda_df.drop(columns=eda_df.filter(regex=regex_pattern).columns)

In [10]:
# Average of last 6 months
average_6_values = calculate_average_last_6_months(eda_df)

new_column_names = {col: f"{col}_Ave_6" for col in average_6_values.index if col!= 'customer_ID'}

average_6_values = average_6_values.rename(index=new_column_names)

average_6_values

P_2_Ave_6       0.645674
D_39_Ave_6      0.167517
B_1_Ave_6       0.132256
B_2_Ave_6       0.604573
R_1_Ave_6       0.093123
                  ...   
D_142_Ave_6     0.391815
D_143_Ave_6     0.179578
D_144_Ave_6     0.051805
D_145_Ave_6     0.063218
target_Ave_6    0.253945
Length: 178, dtype: float64

In [11]:
# Average of last 12 months
average_12_values = calculate_average_last_12_months(eda_df)

new_column_names = {col: f"{col}_Ave_12" for col in average_12_values.index if col!= 'customer_ID'}

average_12_values = average_12_values.rename(index=new_column_names)

average_12_values

P_2_Ave_12       0.655332
D_39_Ave_12      0.154940
B_1_Ave_12       0.124777
B_2_Ave_12       0.620399
R_1_Ave_12       0.080536
                   ...   
D_142_Ave_12     0.390749
D_143_Ave_12     0.178400
D_144_Ave_12     0.052090
D_145_Ave_12     0.062331
target_Ave_12    0.247798
Length: 178, dtype: float64

In [12]:
# Calculate minimum value
min_6_values = calculate_min_last_6_months(eda_df)

new_column_names = {col: f"{col}_min_6" for col in min_6_values.index if col!= 'customer_ID'}

min_6_values = min_6_values.rename(index=new_column_names)

min_6_values


P_2_min_6      -4.205811e-01
D_39_min_6      9.052854e-09
B_1_min_6      -8.993963e-01
B_2_min_6       5.485064e-08
R_1_min_6       1.065542e-08
                    ...     
D_142_min_6    -1.422761e-02
D_143_min_6     2.565179e-08
D_144_min_6     4.701217e-09
D_145_min_6     2.811854e-09
target_min_6    0.000000e+00
Length: 178, dtype: float64

In [14]:
# Calculate maximum value
max_9_values = calculate_max_last_9_months(eda_df)

new_column_names = {col: f"{col}_max_9" for col in max_9_values.index if col!= 'customer_ID'}

max_9_values = max_9_values.rename(index=new_column_names)

max_9_values

P_2_max_9       1.010000
D_39_max_9      5.268649
B_1_max_9       1.324059
B_2_max_9       1.010000
R_1_max_9       3.006102
                  ...   
D_142_max_9     2.091407
D_143_max_9     1.010000
D_144_max_9     1.343331
D_145_max_9     4.827630
target_max_9    1.000000
Length: 178, dtype: float64

In [15]:
# Calculate total value
sum_3_values = calculate_sum_of_3_months(eda_df)

new_column_names = {col: f"{col}_sum_3" for col in sum_3_values.index if col!= 'customer_ID'}

sum_3_values = sum_3_values.rename(index=new_column_names)

sum_3_values

P_2_sum_3       173460.410369
D_39_sum_3       50358.957004
B_1_sum_3        37254.089682
B_2_sum_3       163096.167642
R_1_sum_3        27327.040214
                    ...      
D_142_sum_3      18441.322821
D_143_sum_3      48197.847675
D_144_sum_3      14054.392180
D_145_sum_3      17013.017700
target_sum_3     69922.000000
Length: 178, dtype: float64

In [23]:
# Check April 2018 data
april_2018_data = eda_df[(eda_df['S_2'].dt.year == 2018) & (eda_df['S_2'].dt.month == 4)]
april_2018_data.head()

,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,...,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145,target


,min,max,range
customer_ID,,,
000098081fde4fd64bc4d503a5d6f86a0aedc425c96f5235f98b0f47c9d7d8d4,2017-03-11,2018-03-12,366 days
000445609ff2a39d2dd02484899affa5696210a95f6869f26390bd26eeb3b651,2017-03-10,2018-03-17,372 days
0004837f0c785928a29a6f83f70f4a1c54caec483a773ff4b5b317ac251abda0,2017-03-01,2018-03-02,366 days
0004ec03ca1ab2adb9aa260c61ba5dce8185e19d3ab704029f989240c733b6d0,2017-03-09,2018-03-08,364 days
00050d84c6d26e26cd2b18c3eed83d3130c270e2361470ff272f9409103d067f,2017-03-14,2018-03-22,373 days
...,...,...,...
fffe2bc02423407e33a607660caeed076d713d8a5ad32321530e92704835da88,2017-08-30,2018-03-17,199 days
ffff518bb2075e4816ee3fe9f3b152c57fc0e6f01bf7fdd3e5b57cfcbee30286,2017-03-22,2018-03-22,365 days
ffff9984b999fccb2b6127635ed0736dda94e544e67e026eee4d20f680639ff6,2017-03-16,2018-03-07,356 days


P_2         0.371958
D_39        0.004968
B_1         0.002395
B_2         0.811319
R_1         0.005772
              ...   
D_68_3.0    0.000000
D_68_4.0    0.000000
D_68_5.0    1.000000
D_68_6.0    0.000000
D_68_nan    0.000000
Length: 233, dtype: float64

[[0          False
  1          False
  2          False
  3          False
  4          False
             ...  
  1107077    False
  1107078    False
  1107079    False
  1107080    False
  1107081    False
  Name: customer_ID, Length: 1107082, dtype: bool]]